### Dataset exploration

In [ ]:
# import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
# from fastai.collab import CollabDataLoaders, collab_learner
# import torch
# from fastai.collab import CollabFilterDataset
# from fastai.collab import get_cv_idxs
# from fastai.collab import CollabLearner
# from torch import optim
# import tensorflow as tf
# import keras_core as keras
# import keras_nlp
# from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
# import seaborn as sns
# import neptune.new as neptune
# import nlpaug.augmenter.word as naw
# from tqdm import tqdm
# from sklearn.utils import shuffle
# import matplotlib.pyplot as plt

In [ ]:
# df_train = pd.read_csv("datasets/amazon-m2/products_train.csv")
# df_test = pd.read_csv("datasets/amazon-m2/sessions_train.csv")
# ratings = pd.read_csv("datasets/movieLenz-25ml/ratings.csv")
# ratings = pd.read_csv("datasets/ml-100k/ml-100k")
# df_test = pd.read_csv("datasets/movieLenz-25ml/movies.csv")
# print('Training Set Shape = {}'.format(ratings.shape))
# print('Training Set Memory Usage = {:.2f} MB'.format(ratings.memory_usage().sum() / 1024**2))
# print('Test Set Shape = {}'.format(df_test.shape))
# print('Test Set Memory Usage = {:.2f} MB'.format(df_test.memory_usage().sum() / 1024**2))

In [ ]:
# ratings.head()


In [ ]:
# df_test.head()

# locale_counts = df_test['locale'].value_counts()
# print(locale_counts)
# df_test.head()

In [4]:
torch.get_device_name(0)

NameError: name 'torch' is not defined

In [ ]:
# # ratings = pd.read_csv('ratings_small.csv') # loading data from csv
# """
# ratings_small.csv has 4 columns - userId, movieId, ratings, and timestammp
# it is most generic data format for CF related data
# """
#
# val_indx = get_cv_idxs(len(ratings))  # index for validation set
# wd = 2e-4 # weight decay
# n_factors = 50 # n_factors - dimension of embedding matrix (D)
#
# # data loader
# cf = CollabFilterDataset.from_csv(path, 'ratings_small.csv', 'userId', 'movieId', 'rating')
#
# # learner initializes model object
# learn = cf.get_learner(n_factors, val_indx, bs=64, opt_fn=optim.Adam)
#
# # fitting model with 1e-2 learning rate, 2 epochs,
# # (1 cycle length and 2 cycle multiple for learning rate scheduling)
# learn.fit(1e-2,2, wds = wd, cycle_len=1, cycle_mult=2)

In [ ]:
# x = ratings.drop([‘rating’],axis=1)
# y = ratings[‘rating’].astype(np.float32)
# data = ColumnarModelData.from_data_frame(path, val_indx, x, y, [‘userId’, ‘movieId’], 64)

In [ ]:
# from torch import optim
# from fastai.tabular.all import TabularDataLoaders
# from fastai.collab import index
# from fastai.collab import *
# from fastai.metrics import

In [ ]:
# # x = ratings.drop(['rating'],axis=1)
# # y = ratings['rating'].astype(np.float32)
#
# dls = CollabDataLoaders.from_df(ratings, bs=64)
# dls.show_batch()


In [ ]:
# x,y = dls.one_batch()
# model = EmbeddingDotBias(50, len(dls.classes['userId']), len(dls.classes['movieId']), y_range=(0,5)
#                         ).to(x.device)
# out = model(x)
# assert (0 <= out).all() and (out <= 5).all()

In [ ]:
# learn = collab_learner(dls, y_range=(0,5))
# learn.fit_one_cycle(1)


In [ ]:
# lr = 1e-2
#
# # Reduce the weight decay
# wd = 0.01
#
# # Reduce the number of epochs
# num_epochs = 3
#
# learn = collab_learner(dls, n_factors=10, y_range=(0.5, 5.5))
# learn.fit_one_cycle(num_epochs, lr, wd=wd)

In [3]:
ratings = pd.read_csv('datasets/ml-100k/ml-100k/u.data', delimiter='\t', header=None,
                      usecols=(0,1,2), names=['user','movie','rating'])
ratings.head()

,user,movie,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [ ]:
movies = pd.read_csv('datasets/ml-100k/ml-100k/u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies.head()

In [ ]:
ratings = ratings.merge(movies)
ratings.head()

In [ ]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)


In [ ]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))
learn.fit_one_cycle(5, 5e-3, wd=0.1)


In [ ]:
from torchmetrics.retrieval import RetrievalNormalizedDCG
from torchmetrics import RetrievalMRR


In [ ]:
g = ratings.groupby('title')['rating'].count()
top_movies = g.sort_values(ascending=False).index.values[:1000]
top_movies[:10]

In [ ]:
movie_bias = learn.model.bias(top_movies, is_item=True)
movie_bias.shape